# ⭐️ Transformer Architecture
* ⭐️ encoder: 언어 이해
* ⭐️ decoder: 언어 생성
* ⭐️ 병렬 연산 O: 모든 입력을 동시에 처리
* 대부분 LLM(Large Language Model, 대규모 언어 모델)에서 사용: 딥러닝 기반
* 구글 논문: Attention is All you need.
* self-attention: 입력된 문장 내의 각 단어가 서로 어떤 관련이 있는지 계산.
---
### cf. RRN(Recurrent Neural Network, 순환신경망)
* 입력을 순차적⭐️으로 처리. 즉 병렬 처리 X
---

## 1️⃣ 토큰화(tokenization): 텍스트를 적절한 단위로 잘라 숫자 ID를 부여.
* 사전(vocabulary): 토큰과 매칭된 숫자 ID 기록.
* ex) 한글: (자음과 모음 < 음절 < 단어) => 작은 단위 < 큰단위
  * 큰 단위 기준으로 자른다면
    * (-) OOV(Out Of Vocabulary, 사전에 없는 단어): 사전에 없으면 처리하지 못하는 문제.
    * (-) 텍스트에 등장하는 단어 수만큼 토큰 ID가 필요하기 때문에 사전의 크기가 커짐.
    * (+) 텍스트의 의미가 잘 유지됨.

* ⭐️ subword(多): 등장 빈도에 따라 토큰화 단위 결정.
  * 자주 나오는 단어는 그대로 의미 유지, 드물게 나오는 단어는 작게 나눠 사전 크기가 커지지 않도록 함.
  * 한글의 경우 보통 음절(ex. 특수문자, 이모티콘)과 단어(ex. 유명인 이름)로 토큰화.

In [1]:
# 띄어쓰기 단위로 분리 (실습 편의상, 실무에서는 subword 주로 사용.)
input_text = '나는 최근 미국 여행을 다녀왔다'
input_text_list = input_text.split()
print(input_text_list)

['나는', '최근', '미국', '여행을', '다녀왔다']


In [2]:
# 토큰 딕셔너리
str2idx = {word: idx for idx, word in enumerate(input_text_list)}
print(str2idx)

{'나는': 0, '최근': 1, '미국': 2, '여행을': 3, '다녀왔다': 4}


In [3]:
idx2str = {idx: word for idx, word in enumerate(input_text_list)}
print(idx2str)

{0: '나는', 1: '최근', 2: '미국', 3: '여행을', 4: '다녀왔다'}


In [4]:
input_ids = [str2idx[word] for word in input_text_list]
print(input_ids)

[0, 1, 2, 3, 4]


## 2️⃣ 토큰 embedding: 데이터 -> vector(숫자 집합) 변환 (의미가 담김)


In [5]:
import torch
import torch.nn as nn

# 토큰 ID 1개 => 16차원의 벡터로 변환.
embedding_dim = 16
embed_layer = nn.Embedding(len(str2idx), embedding_dim) # 임베딩 층: 임의의 숫자 집합으로 변환(의미 X), 실무에선 이 층도 학습시켜서 의미를 담음.
print(embed_layer)

Embedding(5, 16)


In [6]:
token_embeddings = embed_layer(torch.tensor(input_ids))
print(token_embeddings)

tensor([[ 0.1160, -0.7262, -1.3192,  0.8191,  0.7993, -0.7065,  0.4225,  0.2210,
          1.2648, -0.0686, -1.2311,  0.4734,  0.9042,  0.3324,  1.0393, -0.8071],
        [ 0.9720,  0.5177, -0.3043, -2.1011, -0.0873,  0.5373,  0.2749,  1.8464,
         -0.9620,  0.1327, -1.5265,  0.8904,  0.0932, -2.2419, -0.9434, -0.4522],
        [ 0.1612, -0.8149, -1.4896, -0.9075,  1.1382, -0.1885,  1.0224,  0.4806,
         -0.3866, -0.6041,  0.3711,  2.1835, -0.0756,  2.0240,  0.3116, -0.7648],
        [ 1.7334,  1.7280,  0.6350,  0.1689, -1.3094, -3.5773,  0.3951, -0.2788,
          1.8887, -0.1501, -1.0158, -0.5792, -0.8352, -0.1427,  0.4078, -1.5921],
        [-1.5153, -0.5584, -1.8897, -0.3588, -1.6828, -1.3027,  0.0464,  1.0681,
         -0.5563,  1.7615,  0.5348, -0.6170, -0.5167,  0.0480,  0.9180,  0.8601]],
       grad_fn=<EmbeddingBackward0>)


In [7]:
token_embeddings = token_embeddings.unsqueeze(0)
print(token_embeddings)

tensor([[[ 0.1160, -0.7262, -1.3192,  0.8191,  0.7993, -0.7065,  0.4225,
           0.2210,  1.2648, -0.0686, -1.2311,  0.4734,  0.9042,  0.3324,
           1.0393, -0.8071],
         [ 0.9720,  0.5177, -0.3043, -2.1011, -0.0873,  0.5373,  0.2749,
           1.8464, -0.9620,  0.1327, -1.5265,  0.8904,  0.0932, -2.2419,
          -0.9434, -0.4522],
         [ 0.1612, -0.8149, -1.4896, -0.9075,  1.1382, -0.1885,  1.0224,
           0.4806, -0.3866, -0.6041,  0.3711,  2.1835, -0.0756,  2.0240,
           0.3116, -0.7648],
         [ 1.7334,  1.7280,  0.6350,  0.1689, -1.3094, -3.5773,  0.3951,
          -0.2788,  1.8887, -0.1501, -1.0158, -0.5792, -0.8352, -0.1427,
           0.4078, -1.5921],
         [-1.5153, -0.5584, -1.8897, -0.3588, -1.6828, -1.3027,  0.0464,
           1.0681, -0.5563,  1.7615,  0.5348, -0.6170, -0.5167,  0.0480,
           0.9180,  0.8601]]], grad_fn=<UnsqueezeBackward0>)


In [8]:
token_embeddings.shape # 문장 1개, 토큰 5개(각 임베딩 16차원)

torch.Size([1, 5, 16])

## 3️⃣ 위치 인코딩(position encoding)
* 텍스트에서 순서는 매우 중요한 정보

In [9]:
# 절대적(absolute) 위치 인코딩: 토큰의 위치에 따라 고정된 임베딩을 더함. (<-> 상대적(relative) 위치 인코딩)
max_position = 12
position_embed_layer = nn.Embedding(max_position, embedding_dim) # 위치 인코딩 층
print(position_embed_layer)

Embedding(12, 16)


In [10]:
position_ids = torch.arange(len(input_ids), dtype=torch.long).unsqueeze(0)
print(position_ids)

tensor([[0, 1, 2, 3, 4]])


In [11]:
position_encodings = position_embed_layer(position_ids)
print(position_encodings)

tensor([[[-0.2843, -0.9143,  0.7962, -0.8312, -0.7476,  0.3424,  1.6838,
          -0.3676, -0.9341, -0.9542,  1.0027,  1.6564, -0.5288, -0.8302,
          -2.2409, -2.7816],
         [-0.8405,  0.0154, -0.2683,  0.5512, -0.6088,  0.5670, -0.9249,
           1.4955, -0.5395, -0.9027,  2.0104,  0.5816, -0.8601, -1.2781,
          -0.7299,  1.5986],
         [ 0.3015,  1.2039,  1.6438, -1.2443,  0.3622, -0.5476,  0.2646,
          -0.4422, -0.8375, -1.1003, -0.2834, -1.3641, -0.5535, -0.7492,
           0.3403,  1.3708],
         [-1.8227, -0.5611,  1.7791, -0.7227, -1.1062,  1.9822,  0.6457,
          -1.1013,  0.2705, -0.3542, -1.3914, -1.3609, -1.7423, -0.3242,
          -0.4138,  0.6098],
         [-0.7928, -0.1854,  0.1147, -1.7375, -1.7869, -0.3057, -0.4513,
           0.8481,  0.5399, -1.6301, -1.5184,  0.1195, -0.3041,  0.0882,
           0.4547, -0.2332]]], grad_fn=<EmbeddingBackward0>)


In [12]:
input_embeddings = token_embeddings + position_encodings
print(input_embeddings) # 모델에 입력할 최종 임베딩

tensor([[[-0.1682, -1.6404, -0.5230, -0.0121,  0.0516, -0.3641,  2.1064,
          -0.1466,  0.3308, -1.0228, -0.2283,  2.1298,  0.3755, -0.4978,
          -1.2016, -3.5887],
         [ 0.1315,  0.5331, -0.5726, -1.5499, -0.6960,  1.1043, -0.6500,
           3.3419, -1.5016, -0.7700,  0.4838,  1.4720, -0.7668, -3.5200,
          -1.6733,  1.1464],
         [ 0.4627,  0.3890,  0.1542, -2.1518,  1.5004, -0.7361,  1.2870,
           0.0385, -1.2241, -1.7044,  0.0877,  0.8194, -0.6290,  1.2748,
           0.6519,  0.6060],
         [-0.0893,  1.1668,  2.4141, -0.5538, -2.4156, -1.5950,  1.0408,
          -1.3801,  2.1592, -0.5043, -2.4071, -1.9401, -2.5776, -0.4670,
          -0.0060, -0.9824],
         [-2.3080, -0.7438, -1.7750, -2.0962, -3.4697, -1.6084, -0.4049,
           1.9162, -0.0164,  0.1314, -0.9836, -0.4975, -0.8209,  0.1361,
           1.3727,  0.6269]]], grad_fn=<AddBackward0>)


In [13]:
input_embeddings.shape

torch.Size([1, 5, 16])

## 4️⃣ Attention: 사람이 단어 사이의 관계를 고민하는 과정을 딥러닝 모델이 수행할 수 있도록 모방.
* 사람: 맥락을 반영해 단어를 재해석 => ⭐️ 가중치를 사용해 토큰 간 관계를 계산해서 관련이 깊은 단어와 그렇지 않은 단어를 구분
1. query: 검색어
2. key: 문서가 가진 특징 (ex. 제목, 저자 이름, 문장 속의 각 단어)
3. value: 원하는 값

In [14]:
head_dim = 16

weight_q = nn.Linear(embedding_dim, head_dim) # 선형 층
weight_k = nn.Linear(embedding_dim, head_dim)
weight_v = nn.Linear(embedding_dim, head_dim)

querys = weight_q(input_embeddings)
keys = weight_k(input_embeddings)
values = weight_v(input_embeddings)

print(querys) # (1, 5, 16)
print(keys)
print(values)

tensor([[[ 1.3045e+00, -3.9089e-01,  1.3074e+00,  1.9847e-01,  2.6002e-01,
           4.9307e-01,  1.1281e-01, -4.0106e-01,  4.4259e-01,  7.1924e-01,
          -6.9390e-01,  7.2495e-01,  4.0594e-01,  5.3339e-01, -1.3246e+00,
          -1.9372e-01],
         [ 2.1203e-01, -2.4054e+00, -1.0225e+00,  1.0628e+00,  1.9424e-01,
          -3.9157e-01, -3.9018e-01, -2.3384e-01, -1.1760e+00, -6.2021e-01,
           6.7474e-02,  1.3634e+00, -5.4305e-01,  1.2736e+00, -1.8374e+00,
           4.8560e-01],
         [-4.6478e-01, -3.0420e-01, -4.2859e-01, -3.1556e-01,  3.0028e-01,
          -3.9897e-01,  1.0369e+00,  3.7221e-01,  1.4550e-01,  1.0903e+00,
          -3.0956e-02,  3.0964e-01, -3.2314e-01,  2.7083e-01,  2.1801e-01,
           3.0985e-01],
         [ 1.8702e-01,  1.2337e+00,  2.7763e-01, -1.3426e+00,  4.6012e-01,
           1.2876e+00,  9.3376e-01, -2.5120e-01,  1.0356e+00,  1.5255e-04,
           3.5870e-01, -1.3668e+00,  1.2642e-01,  7.1631e-01,  1.1488e+00,
           1.1959e-01],
    

In [15]:
# 스케일 점곱 어텐션: 1개의 어텐션 연산 수행.
from math import sqrt
import torch.nn.functional as F

def compute_attention(querys, keys, values, is_causal=False):
    dim_k = querys.size(-1) # 16
    scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
    if is_causal: # 디코더 - 마스크 어텐션: 작성해야 하는 텍스트를 미리 확인하게 되는 문제 해결. 즉, 미래 시점의 토큰을 제거하여 생성된 토큰까지만 확인할 수 있도록 마스크를 추가.
        query_length = querys.size(-2)
        key_length = keys.size(-2)
        temp_mask = torch.ones(query_length, key_length, dtype=torch.bool).tril(diagonal=0) # 모두 1인 행렬에 대각선 아래 부분만 1로 유지되고, 나머지는 음의 무한대로 변경해 마스크를 생성.
        scores = scores.masked_fill(temp_mask == False, float('-inf')) # 행렬의 대각선 아래 부분만 어텐션 스코어가 남고, 위쪽은 음의 무한대가 된다.
    weights = F.softmax(scores, dim=-1) # 총 합이 1 / 음의 무한대인 대각선 윗부분은 가중치가 0이 됨.
    return weights @ values

after_attention_embeddings = compute_attention(querys, keys, values) # 입력, 출력 형태 동일
print(after_attention_embeddings.shape)

torch.Size([1, 5, 16])


In [16]:
class AttentionHead(nn.Module):
    def __init__(self, token_embed_dim, head_dim, is_causal=False):
        super().__init__()
        self.is_causal = is_causal
        self.weight_q = nn.Linear(token_embed_dim, head_dim)
        self.weight_k = nn.Linear(token_embed_dim, head_dim)
        self.weight_v = nn.Linear(token_embed_dim, head_dim)

    def forward(self, querys, keys, values):
        outputs = compute_attention(
            self.weight_q(querys),
            self.weight_k(keys),
            self.weight_v(values),
            is_causal=self.is_causal
        )
        return outputs

attention_head = AttentionHead(embedding_dim, embedding_dim)
after_attention_embeddings = attention_head(input_embeddings, input_embeddings, input_embeddings)
print(after_attention_embeddings.shape)

torch.Size([1, 5, 16])


In [17]:
# 멀티 헤드 어텐션: 여러 어텐션 연산을 헤드의 수(n_head)만큼 동시에 수행, 단어 사이의 관계를 계산.
class MultiheadAttention(nn.Module):
    def __init__(self, token_embed_dim, d_model, n_head, is_causal=False):
        super().__init__()
        self.n_head = n_head
        self.is_causal = is_causal
        self.weight_q = nn.Linear(token_embed_dim, d_model)
        self.weight_k = nn.Linear(token_embed_dim, d_model)
        self.weight_v = nn.Linear(token_embed_dim, d_model)
        self.concat_linear = nn.Linear(d_model, d_model)

    def forward(self, querys, keys, values):
        B, T, C = querys.size()

        querys = self.weight_q(querys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        keys = self.weight_k(keys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        values = self.weight_v(values).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)

        attention = compute_attention(querys, keys, values, is_causal=self.is_causal) # n_head번의 스케일 점곱 어텐션
        output = attention.transpose(1, 2).contiguous().view(B, T, C) # 어텐션 결과를 연결
        output = self.concat_linear(output) # 마지막 선형 층
        return output

n_head = 4
mh_attention = MultiheadAttention(embedding_dim, embedding_dim, n_head)
after_attention_embeddings = mh_attention(input_embeddings, input_embeddings, input_embeddings)
print(after_attention_embeddings.shape)

torch.Size([1, 5, 16])


## 5️⃣ 정규화(normalization): 딥러닝 모델의 모든 입력 변수가 비슷한 범위와 분포를 갖도록 조정하면 각 입력 변수의 중요성을 공정하게 반영하여 더 정확한 예측을 할 수 있음.
* 1. 층(layer) 정규화
    - 각 층의 입력 데이터 분포를 균일하게 만들어 딥러닝 모델이 원할하게 학습되도록 함.
    - 자연어 처리
    - Transformer Architecture
    - 층과 층 사이에 정규화
    - 각 토큰 임베딩의 평균과 표준편차를 구함.
    - 사전 정규화(pre-norm)多: 먼저 층 정규화를 적용하고 어텐션과 피드 포워드 층을 통과했을 때 학습이 더 안정적. (cf. 사후 정규화(post-norm))

* 2. batch 정규화: 이미지 처리

In [18]:
# 층 정규화, 사전 정규화
norm = nn.LayerNorm(embedding_dim) # 층 정규화 레이어
norm_x = norm(input_embeddings) # 정규화된 임베딩
norm_x.shape

torch.Size([1, 5, 16])

In [19]:
norm_x.mean(dim=-1).data, norm_x.std(dim=-1).data # 평균, 표준편차

(tensor([[-1.4901e-08, -2.9802e-08, -1.4901e-08,  0.0000e+00,  2.9802e-08]]),
 tensor([[1.0328, 1.0328, 1.0328, 1.0328, 1.0328]]))

In [20]:
# feed forward layer, 완전 연결 층(fully connected layer)
# > 입력 텍스트 전체를 이해.
# > 데이터의 특징을 학습.
class PreLayerNormFeedForward(nn.Module):
    def __init__(self, d_model, dim_feedforward, dropout):
        super().__init__()
        self.linear1 = nn.Linear(d_model, dim_feedforward) # 선형 층
        self.linear2 = nn.Linear(dim_feedforward, d_model) # 선형 층
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.activation = nn.GELU()
        self.norm = nn.LayerNorm(d_model) # 층 정규화

    def forward(self, src):
        x = self.norm(src)
        x = x + self.linear2(self.dropout1(self.activation(self.linear1(x))))
        x = self.dropout2(x)
        return x

## 6️⃣ 인코더
* 자연어 이해(Natural Language Understanding, NLU)
* 양방향

In [21]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, n_head, dim_feedforward, dropout):
        super().__init__()
        self.attn = MultiheadAttention(d_model, d_model, n_head)
        self.norm1 = nn.LayerNorm(d_model) # 층 정규화
        self.dropout1 = nn.Dropout(dropout)
        self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout)

    def forward(self, src):
        norm_x = self.norm1(src)
        attn_output = self.attn(norm_x, norm_x, norm_x)
        x = src + self.dropout1(attn_output) # 잔차 연결(residual connection): 안정적인 학습이 가능하도록 도와줌.
        return self.feed_forward(x)


import copy
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)]) # 깊은 복사

class TransformerEncoder(nn.Module): # 인코더 층을 N번 반복 수행.
    def __init__(self, encoder_layer, num_layers):
        super().__init__()
        self.layers = get_clones(encoder_layer, num_layers)
        self.num_layers = num_layers
        self.norm = norm

    def forward(self, src):
        output = src
        for mod in self.layers:
            output = mod(output)
        return output


## 7️⃣ 디코더
* 자연어 생성(Natural Language Generation, NLG)
* 단방향
* 사람이 글을 쓸 때 앞 단어부터 순차적으로 작성하는 것처럼 트랜스포머 모델도 앞에서 생성한 토큰을 기반으로 다음 토큰을 생성.
> 순차적으로 생성: 인과적(causal) == 자기 회귀적(auto-regressive)

In [22]:
class TransformerDecoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        self.self_attn = MultiheadAttention(d_model, d_model, nhead)
        self.multihead_attn = MultiheadAttention(d_model, d_model, nhead)
        self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, tgt, encoder_output, is_causal=True):
        x = self.norm1(tgt)
        x = x + self.dropout1(self.self_attn(x, x, x, is_causal=is_causal))

        # ⭐️cross attention: 인코더의 결과를 디코더가 활용
        x = self.norm2(x)
        x = x + self.dropout2(self.multihead_attn(x, encoder_output, encoder_output))

        return self.feed_forward(x)


class TransformerDecoder(nn.Module):
    def __init__(self, decoder_layer, num_layers):
        super().__init__()
        self.layers = get_clones(decoder_layer, num_layers) # 디코더 층을 N번 반복.
        self.num_layers = num_layers

    def forward(self, tgt, src):
        output = tgt
        for mod in self.layers:
            output = mod(output, src)
        return output